In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

2025-03-30 14:30:46.335969: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-30 14:30:46.387652: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-30 14:30:46.387686: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-30 14:30:46.389269: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-30 14:30:46.400069: I tensorflow/core/platform/cpu_feature_guar

GPU is available


In [2]:
# Read from CSV instead of pickle
dataset = pd.read_csv('cleaned_profiles.csv')
dataset

,Intro,Full Name,Workplace,Location,Connections,Photo,Followers,About,Experiences,Number of Experiences,...,Number of Scores,Languages,Number of Languages,Organizations,Number of Organizations,Interests,Number of Interests,Activities,Number of Activities,Label
0,"{'Full Name': 'chenxia (polly) Pei', 'Workplac...",chenxia (polly) Pei,Jiangsu Junyao mainly offer services to cement...,"Wuxi, Jiangsu, China",500,No,717,NaN,"{'0': {'Role': 'Sales', 'Workplace': 'Jiangsu ...",2,...,0,{},0,{},0,"{'0': {'Interest': 'Trina Solar', 'ID': '69648...",4,{'chenxia-pei-80177594': {'Full Name': 'chenxi...,1,1
1,"{'Full Name': 'NEHA CHANDOK', 'Workplace': 'So...",NEHA CHANDOK,Software Analyst,"Noida, Uttar Pradesh, India",500,No,1340,NaN,"{'0': {'Role': 'Software Analyst', 'Workplace'...",1,...,0,"{'Language 0': 'Bangla', 'Language 1': 'Englis...",3,{},0,"{'0': {'Interest': 'Mohamed El-Erian', 'ID': '...",8,{},0,1
2,"{'Full Name': 'Mounika Mungamuri', 'Workplace'...",Mounika Mungamuri,Senior Consultant at Infosys,"Hyderabad, Telangana, India",7,Yes,7,NaN,"{'0': {'Role': 'Senior Consultant', 'Workplace...",2,...,0,{},0,{},0,{},0,{},0,1
3,"{'Full Name': 'Katarina Djuric', 'Workplace': ...",Katarina Djuric,--,"Belgrade, Serbia",0,Yes,0,NaN,"{'0': {'Role': 'Instructor', 'Workplace': 'GE'...",1,...,0,{},0,{},0,"{'0': {'Interest': 'GE', 'ID': '1015', 'Catego...",1,{},0,1
4,"{'Full Name': 'Rachel Lally', 'Workplace': '--...",Rachel Lally,--,"Dublin, County Dublin, Ireland",61,Yes,61,NaN,"{'0': {'Role': 'Bartender', 'Workplace': ""O'Su...",1,...,1,{},0,{},0,"{'0': {'Interest': 'Richard Branson', 'ID': 'r...",3,"{'garyltravis': {'Full Name': 'Gary Travis', '...",6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,"{'Full Name': 'Emily Williams', 'Workplace': ""...",Emily Williams,Market Research Analyst at L'Oreal,"New York City, New York",106,No,717,{},Market Research Analyst Elizabeth Arden Jan 20...,3,...,0,{},0,{},0,{},3,{},4,11
3596,"{'Full Name': 'Sarah Johnson', 'Workplace': 'D...",Sarah Johnson,Director of Marketing Strategy at Acme Inc.,"San Francisco, California",102,No,6,{},Director of Marketing Acme Inc. January 2018-P...,5,...,0,{},0,{},0,{},5,{},3,11
3597,"{'Full Name': 'Sarah Johnson', 'Workplace': 'M...",Sarah Johnson,Manager at Acme Inc.,"New York City, New York",435,No,10,A results-driven and experienced professional ...,{},0,...,0,{},0,{},0,{},6,{},0,11
3598,"{'Full Name': 'Emma Williams', 'Workplace': 'M...",Emma Williams,Manager and Director at ABC Inc.,"Seattle, Washington",280,No,34,With over 10 years of experience in operations...,Operations Manager ABC Inc. January 2015- Pres...,1,...,0,English Spanish,2,{},0,{},4,{},4,11


In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, confusion_matrix, brier_score_loss
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
import pickle
import os
import seaborn as sns
from scipy.stats import pearsonr, spearmanr

def get_classifier(classifier_name, random_state=42):
    """Return classifier instance based on name"""
    if classifier_name == 'svc':
        return SVC(random_state=random_state, probability=True)
    elif classifier_name == 'logistic_regression':
        return LogisticRegression(random_state=random_state, max_iter=1000)
    elif classifier_name == 'catboost':
        return CatBoostClassifier(random_state=random_state, verbose=False)

def calculate_metrics(y_true, y_pred, y_prob=None):
    """Calculate F1, FAR, and FRR, plus Brier score if probabilities provided"""
    metrics = {}
    f1 = f1_score(y_true, y_pred, average='weighted')
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    frr = fp / (fp + tn)
    far = fn / (fn + tp)
    
    metrics = {'f1_score': f1, 'far': far, 'frr': frr}
    
    # Calculate Brier score if probabilities provided
    if y_prob is not None:
        brier = brier_score_loss(y_true, y_prob)
        metrics['brier_score'] = brier
        
    return metrics

def create_results_dir():
    """Create results directory if it doesn't exist"""
    results_dir = 'brier_far_analysis'
    os.makedirs(results_dir, exist_ok=True)
    return results_dir

class BrierFARAnalysis:
    def __init__(self, model_name):
        self.model_name = model_name
        self.results_dir = create_results_dir()
        self.scaler = StandardScaler()
        
        # Define classifiers with fixed parameters
        self.classifiers = {
            'svc': get_classifier('svc'),
            'logistic_regression': get_classifier('logistic_regression'),
            'catboost': get_classifier('catboost')
        }
    
    def _get_data_splits(self, embedding_file, random_state=42):
        """Prepare all data splits with 70-30 ratio"""
        # Load embeddings
        try:
            self.emb = pd.read_csv(embedding_file)
            print(f"Loaded embeddings from {embedding_file}")
        except FileNotFoundError:
            print(f"Warning: Could not find {embedding_file}. Using a placeholder path.")
            self.emb = pd.read_csv(f'embeddings_output/{self.model_name}_PCA_150_components.csv')
        
        # CHANGE HERE: Use dataset instead of trying to extract labels from embeddings
        # We need to load the 'dataset' that has the Label column
        # Assuming a consistent filename for the dataset, but you may need to adjust this path
        # try:
        #     dataset = pd.read_csv('your_dataset.csv')  # Update with your actual dataset path
        #     print(f"Loaded dataset with labels")
        # except FileNotFoundError:
        #     print(f"Error: Could not find dataset with labels. Please provide the correct path.")
        #     return None
        
        # Use the labels from dataset
        self.genuine_mask = dataset['Label'] == 0
        self.fake_manual_mask = dataset['Label'] == 1
        self.gptv1_mask = dataset['Label'].isin([10, 11])
        self.gptv2_mask = dataset['Label'] == 12
            
        # Get generated profile embeddings file based on current model
        generated_file = f'embeddings_output/{self.model_name}_PCA_150_components_generated.csv'
        
        # Split genuine profiles (70-30)
        X_genuine = self.emb[self.genuine_mask]
        X_train_genuine, X_test_genuine = train_test_split(
            X_genuine, test_size=0.3, random_state=random_state
        )
        
        # Split manual fake profiles (70-30)
        X_fake = self.emb[self.fake_manual_mask]
        X_train_fake, X_test_fake = train_test_split(
            X_fake, test_size=0.3, random_state=random_state
        )
        
        splits = {
            'train': {
                'genuine': X_train_genuine,
                'fake_manual': X_train_fake,
            },
            'test': {
                'genuine': X_test_genuine,
                'fake_manual': X_test_fake,
            }
        }
        
        # Add GPTv1 data if available
        if any(self.gptv1_mask):
            X_gptv1 = self.emb[self.gptv1_mask]
            X_train_gptv1, X_test_gptv1 = train_test_split(
                X_gptv1, test_size=0.3, random_state=random_state
            )
            splits['train']['gptv1'] = X_train_gptv1
            splits['test']['gptv1'] = X_test_gptv1
        else:
            # Use dummy data for GPTv1 if not available
            print("Warning: No GPTv1 data found. Using empty DataFrame.")
            splits['train']['gptv1'] = pd.DataFrame()
            splits['test']['gptv1'] = pd.DataFrame()
            
        # Load GPTv2 profiles if available
        try:
            gptv2_emb = pd.read_csv(generated_file)
            # Only use the first 600 rows if available
            if len(gptv2_emb) > 0:
                gptv2_emb = gptv2_emb.iloc[:min(600, len(gptv2_emb))]
                X_train_gptv2, X_test_gptv2 = train_test_split(
                    gptv2_emb, test_size=0.3, random_state=random_state
                )
                splits['train']['gptv2'] = X_train_gptv2
                splits['test']['gptv2'] = X_test_gptv2
            else:
                # Use dummy data for GPTv2 if file exists but is empty
                print("Warning: GPTv2 file exists but has no data. Using empty DataFrame.")
                splits['train']['gptv2'] = pd.DataFrame()
                splits['test']['gptv2'] = pd.DataFrame()
        except FileNotFoundError:
            # Use dummy data for GPTv2 if file doesn't exist
            print(f"Warning: {generated_file} not found. Using empty DataFrame for GPTv2.")
            splits['train']['gptv2'] = pd.DataFrame()
            splits['test']['gptv2'] = pd.DataFrame()
        
        return splits
        
    def _get_training_data(self, train_data, scenario):
        """Get training data for each scenario"""
        if scenario == 'baseline':
            X_train = pd.concat([
                train_data['genuine'],
                train_data['fake_manual']
            ])
            y_train = pd.Series([0]*len(train_data['genuine']) + [1]*len(train_data['fake_manual']))
            
        elif scenario == 'gptv1_assisted':
            if len(train_data['gptv1']) > 0:
                X_train = pd.concat([
                    train_data['genuine'],
                    train_data['fake_manual'],
                    train_data['gptv1']
                ])
                y_train = pd.Series([0]*len(train_data['genuine']) + 
                                   [1]*(len(train_data['fake_manual']) + len(train_data['gptv1'])))
            else:
                # Fall back to baseline if no GPTv1 data
                print("Warning: No GPTv1 data. Falling back to baseline for training.")
                X_train = pd.concat([
                    train_data['genuine'],
                    train_data['fake_manual']
                ])
                y_train = pd.Series([0]*len(train_data['genuine']) + [1]*len(train_data['fake_manual']))
            
        elif scenario == 'gptv2_assisted':
            if len(train_data['gptv2']) > 0:
                X_train = pd.concat([
                    train_data['genuine'],
                    train_data['fake_manual'],
                    train_data['gptv2']
                ])
                y_train = pd.Series([0]*len(train_data['genuine']) + 
                                   [1]*(len(train_data['fake_manual']) + len(train_data['gptv2'])))
            else:
                # Fall back to baseline if no GPTv2 data
                print("Warning: No GPTv2 data. Falling back to baseline for training.")
                X_train = pd.concat([
                    train_data['genuine'],
                    train_data['fake_manual']
                ])
                y_train = pd.Series([0]*len(train_data['genuine']) + [1]*len(train_data['fake_manual']))
            
        elif scenario == 'gptv1v2_assisted':
            # Check if both GPTv1 and GPTv2 data are available
            if len(train_data['gptv1']) > 0 and len(train_data['gptv2']) > 0:
                X_train = pd.concat([
                    train_data['genuine'],
                    train_data['fake_manual'],
                    train_data['gptv1'],
                    train_data['gptv2']
                ])
                y_train = pd.Series([0]*len(train_data['genuine']) + 
                                   [1]*(len(train_data['fake_manual']) + 
                                       len(train_data['gptv1']) + len(train_data['gptv2'])))
            else:
                # Fall back to baseline if either GPTv1 or GPTv2 data is missing
                print("Warning: Missing GPTv1 or GPTv2 data. Falling back to baseline for training.")
                X_train = pd.concat([
                    train_data['genuine'],
                    train_data['fake_manual']
                ])
                y_train = pd.Series([0]*len(train_data['genuine']) + [1]*len(train_data['fake_manual']))
        
        # Scale the features
        # Ensure X_train has data
        if X_train.shape[0] > 0:
            X_train_scaled = self.scaler.fit_transform(X_train)
            return X_train_scaled, y_train
        else:
            print("Error: No training data available.")
            return np.array([]), pd.Series([])
    
    def _get_test_data(self, test_data, scenario):
        """Get test data for each scenario"""
        if scenario == 'baseline':
            X_test = pd.concat([
                test_data['genuine'],
                test_data['fake_manual']
            ]).reset_index(drop=True)
            y_test = pd.Series([0]*len(test_data['genuine']) + [1]*len(test_data['fake_manual']))
            
        elif scenario == 'gptv1':
            if len(test_data['gptv1']) > 0:
                X_test = pd.concat([
                    test_data['genuine'],
                    test_data['fake_manual'],
                    test_data['gptv1']
                ]).reset_index(drop=True)
                y_test = pd.Series([0]*len(test_data['genuine']) + 
                                  [1]*(len(test_data['fake_manual']) + len(test_data['gptv1'])))
            else:
                # Fall back to baseline if no GPTv1 data
                print("Warning: No GPTv1 data. Falling back to baseline for testing.")
                X_test = pd.concat([
                    test_data['genuine'],
                    test_data['fake_manual']
                ]).reset_index(drop=True)
                y_test = pd.Series([0]*len(test_data['genuine']) + [1]*len(test_data['fake_manual']))
            
        elif scenario == 'gptv2':
            if len(test_data['gptv2']) > 0:
                X_test = pd.concat([
                    test_data['genuine'],
                    test_data['fake_manual'],
                    test_data['gptv2']
                ]).reset_index(drop=True)
                y_test = pd.Series([0]*len(test_data['genuine']) + 
                                  [1]*(len(test_data['fake_manual']) + len(test_data['gptv2'])))
            else:
                # Fall back to baseline if no GPTv2 data
                print("Warning: No GPTv2 data. Falling back to baseline for testing.")
                X_test = pd.concat([
                    test_data['genuine'],
                    test_data['fake_manual']
                ]).reset_index(drop=True)
                y_test = pd.Series([0]*len(test_data['genuine']) + [1]*len(test_data['fake_manual']))
            
        else:  # gptv1v2
            # Check if both GPTv1 and GPTv2 data are available
            if len(test_data['gptv1']) > 0 and len(test_data['gptv2']) > 0:
                X_test = pd.concat([
                    test_data['genuine'],
                    test_data['fake_manual'],
                    test_data['gptv1'],
                    test_data['gptv2']
                ]).reset_index(drop=True)
                y_test = pd.Series([0]*len(test_data['genuine']) + 
                                  [1]*(len(test_data['fake_manual']) + 
                                      len(test_data['gptv1']) + len(test_data['gptv2'])))
            else:
                # Fall back to baseline if either GPTv1 or GPTv2 data is missing
                print("Warning: Missing GPTv1 or GPTv2 data. Falling back to baseline for testing.")
                X_test = pd.concat([
                    test_data['genuine'],
                    test_data['fake_manual']
                ]).reset_index(drop=True)
                y_test = pd.Series([0]*len(test_data['genuine']) + [1]*len(test_data['fake_manual']))
        
        # Scale the features using the training scaler
        # Ensure X_test has data
        if X_test.shape[0] > 0:
            X_test_scaled = self.scaler.transform(X_test)
            return X_test_scaled, y_test
        else:
            print("Error: No test data available.")
            return np.array([]), pd.Series([])
    
    def evaluate_brier_vs_far(self, embedding_file):
        """Evaluate Brier score vs FAR relationship across scenarios and classifiers"""
        splits = self._get_data_splits(embedding_file)
        training_scenarios = ['baseline', 'gptv1_assisted', 'gptv2_assisted', 'gptv1v2_assisted']
        test_scenarios = ['baseline', 'gptv1', 'gptv2', 'gptv1v2']
        results = {}
        
        # For each classifier type
        for clf_name, clf in self.classifiers.items():
            print(f"\nEvaluating {clf_name} classifier with {self.model_name} embeddings...")
            results[clf_name] = {}
            
            # For each training scenario
            for train_scenario in training_scenarios:
                print(f"  Training scenario: {train_scenario}")
                X_train, y_train = self._get_training_data(splits['train'], train_scenario)
                
                # Skip if training data is empty
                if X_train.shape[0] == 0 or len(y_train) == 0:
                    print(f"  Skipping {train_scenario} due to empty training data.")
                    continue
                
                # Train the model
                clf.fit(X_train, y_train)
                
                # Store results for each test scenario
                results[clf_name][train_scenario] = {}
                
                # For each test scenario
                for test_scenario in test_scenarios:
                    print(f"    Testing against: {test_scenario}")
                    X_test, y_test = self._get_test_data(splits['test'], test_scenario)
                    
                    # Skip if test data is empty
                    if X_test.shape[0] == 0 or len(y_test) == 0:
                        print(f"    Skipping {test_scenario} due to empty test data.")
                        continue
                    
                    # Get predictions and probabilities
                    y_pred = clf.predict(X_test)
                    
                    # Get probabilities for class 1 (fake)
                    y_prob = clf.predict_proba(X_test)[:, 1]
                    
                    # Calculate metrics
                    metrics = calculate_metrics(y_test, y_pred, y_prob)
                    results[clf_name][train_scenario][test_scenario] = metrics
        
        return results
    
    def export_results_to_csv(self, results):
        """Export results to CSV for analysis"""
        rows = []
        for clf_name in results.keys():
            for train_scenario in results[clf_name].keys():
                for test_scenario in results[clf_name][train_scenario].keys():
                    metrics = results[clf_name][train_scenario][test_scenario]
                    row = {
                        'model': self.model_name,
                        'classifier': clf_name,
                        'train_scenario': train_scenario,
                        'test_scenario': test_scenario,
                        'brier_score': metrics.get('brier_score', float('nan')),
                        'far': metrics.get('far', float('nan')),
                        'frr': metrics.get('frr', float('nan')),
                        'f1_score': metrics.get('f1_score', float('nan'))
                    }
                    rows.append(row)
        
        df = pd.DataFrame(rows)
        csv_path = f'{self.results_dir}/{self.model_name}_brier_far_results.csv'
        df.to_csv(csv_path, index=False)
        print(f"Exported results to {csv_path}")
        return df
    
    def calculate_correlations(self, df):
        """Calculate correlations between Brier score and FAR"""
        correlations = []
        
        for clf_name in df['classifier'].unique():
            clf_data = df[df['classifier'] == clf_name]
            
            # Calculate Pearson correlation
            pearson_corr, p_value_pearson = pearsonr(
                clf_data['brier_score'].values, 
                clf_data['far'].values
            )
            
            # Calculate Spearman correlation
            spearman_corr, p_value_spearman = spearmanr(
                clf_data['brier_score'].values, 
                clf_data['far'].values
            )
            
            correlations.append({
                'model': self.model_name,
                'classifier': clf_name,
                'pearson_correlation': pearson_corr,
                'pearson_p_value': p_value_pearson,
            })
        
        corr_df = pd.DataFrame(correlations)
        corr_csv_path = f'{self.results_dir}/{self.model_name}_brier_far_correlations.csv'
        corr_df.to_csv(corr_csv_path, index=False)
        print(f"Exported correlation results to {corr_csv_path}")
        return corr_df
    
    def plot_brier_vs_far(self, df, separate_classifiers=True):
        """Create scatter plots of Brier score vs FAR with optimized fonts for 3x2 inch plots"""
        if separate_classifiers:
            # Create separate plots for each classifier
            for clf_name in df['classifier'].unique():
                clf_data = df[df['classifier'] == clf_name]
                
                # Set figure size to exactly 3x2 inches
                plt.figure(figsize=(3, 2), dpi=300)
                
                # Create scatter plot with different colors for each training scenario
                # Use smaller markers due to the small plot size
                for train_scenario in clf_data['train_scenario'].unique():
                    train_data = clf_data[clf_data['train_scenario'] == train_scenario]
                    
                    # Shorten training scenario labels to save space
                    scenario_label = train_scenario
                
                    plt.scatter(
                        train_data['brier_score'], 
                        train_data['far'],
                        label=scenario_label,
                        alpha=0.7,
                        s=20  # Smaller point size
                    )
                
                # Add labels and title with appropriate font sizes
                plt.xlabel('Brier Score', fontsize=7)
                plt.ylabel('FAR', fontsize=7)  # Shortened label
                
                # Use a shorter classifier name in the title
                clf_short = clf_name.replace('logistic_regression', 'LogReg').replace('catboost', 'CatB')
                plt.title(f'{self.model_name} - {clf_short}', fontsize=8)
                
                # Configure tick labels to be smaller
                plt.tick_params(axis='both', which='major', labelsize=6)
                
                # Use a tight grid with smaller lines
                plt.grid(True, linestyle=':', alpha=0.5, linewidth=0.5)
                
                # Make the legend smaller and position it optimally
                plt.legend(fontsize=4, loc='best', framealpha=0.7, 
                          markerscale=0.8, handlelength=1, handletextpad=0.5)
                
                # Add correlation line
                x = clf_data['brier_score'].values
                y = clf_data['far'].values
                
                # Use polyfit to get line of best fit
                m, b = np.polyfit(x, y, 1)
                plt.plot(x, m*x + b, 'r--', linewidth=0.7)
                
                # Calculate Pearson correlation
                pearson_corr, _ = pearsonr(x, y)
                # Add correlation in the corner with smaller font
                plt.annotate(f'r = {pearson_corr:.2f}', 
                            xy=(0.05, 0.90), 
                            xycoords='axes fraction',
                            fontsize=6,
                            bbox=dict(boxstyle="round,pad=0.1", fc="white", ec="gray", alpha=0.7))
                
                # Use tight layout to maximize plot area
                plt.tight_layout(pad=0.1)
                
                # Save the plot at high resolution
                plt_path = f'{self.results_dir}/{self.model_name}_{clf_name}_brier_vs_far.png'
                plt.savefig(plt_path, dpi=300, bbox_inches='tight')
                plt.close()
                print(f"Saved plot to {plt_path}")
        
        # Create a combined plot with all classifiers (keeping this at the original larger size)
        plt.figure(figsize=(6, 4))
        
        # Create scatter plot with different colors for each classifier
        for clf_name in df['classifier'].unique():
            clf_data = df[df['classifier'] == clf_name]
            
            # Shorten classifier names
            clf_short = clf_name.replace('logistic_regression', 'LogReg').replace('catboost', 'CatB')
            
            plt.scatter(
                clf_data['brier_score'], 
                clf_data['far'],
                label=clf_short,
                alpha=0.7,
                s=30
            )
        
        # Add labels and title
        plt.xlabel('Brier Score', fontsize=9)
        plt.ylabel('False Accept Rate (FAR)', fontsize=9)
        plt.title(f'Brier Score vs FAR - {self.model_name}', fontsize=10)
        plt.tick_params(axis='both', which='major', labelsize=8)
        plt.grid(True, linestyle='--', alpha=0.7, linewidth=0.5)
        plt.legend(fontsize=6)
        plt.tight_layout()
        
        # Save the plot
        plt_path = f'{self.results_dir}/{self.model_name}_all_classifiers_brier_vs_far.png'
        plt.savefig(plt_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Saved combined plot to {plt_path}")
    
    def plot_heatmap(self, df):
        """Create a heatmap of Brier score vs attack scenario"""
        # Create a pivot table for the heatmap
        pivot_brier = df.pivot_table(
            index=['classifier', 'train_scenario'],
            columns='test_scenario',
            values='brier_score'
        )
        
        pivot_far = df.pivot_table(
            index=['classifier', 'train_scenario'],
            columns='test_scenario',
            values='far'
        )
        
        # Create heatmap for Brier scores
        plt.figure(figsize=(12, 10))
        sns.heatmap(pivot_brier, annot=True, cmap='YlOrRd', fmt='.4f')
        plt.title(f'Brier Score Heatmap - {self.model_name}', fontsize=12)
        plt.tight_layout()
        
        # Save the plot
        plt_path = f'{self.results_dir}/{self.model_name}_brier_heatmap.png'
        plt.savefig(plt_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Saved Brier score heatmap to {plt_path}")
        
        # Create heatmap for FAR
        plt.figure(figsize=(12, 10))
        sns.heatmap(pivot_far, annot=True, cmap='YlOrRd', fmt='.4f')
        plt.title(f'FAR Heatmap - {self.model_name}', fontsize=12)
        plt.tight_layout()
        
        # Save the plot
        plt_path = f'{self.results_dir}/{self.model_name}_far_heatmap.png'
        plt.savefig(plt_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Saved FAR heatmap to {plt_path}")

def main():
    # Models to analyze
    models = ['roberta', 'modernbert', 'deberta', 'flair']
    
    # Container for all results
    all_results = {}
    all_dfs = []
    
    # Analyze each model
    for model_name in models:
        print(f"\n{'='*30} ANALYZING {model_name.upper()} {'='*30}")
        
        # Initialize analyzer
        analyzer = BrierFARAnalysis(model_name)
        
        # Embedding file path
        embedding_file = f'embeddings_output/{model_name}_PCA_150_components.csv'
        
        # Evaluate Brier vs FAR
        results = analyzer.evaluate_brier_vs_far(embedding_file)
        all_results[model_name] = results
        
        # Export results to CSV
        df = analyzer.export_results_to_csv(results)
        all_dfs.append(df)
        
        # Calculate correlations
        corr_df = analyzer.calculate_correlations(df)
        
        # Create plots
        analyzer.plot_brier_vs_far(df)
        analyzer.plot_heatmap(df)
    
    # Combine all results
    combined_df = pd.concat(all_dfs, ignore_index=True)
    combined_csv_path = 'brier_far_analysis/combined_brier_far_results.csv'
    combined_df.to_csv(combined_csv_path, index=False)
    print(f"Exported combined results to {combined_csv_path}")
    
    # Create combined correlation analysis
    combined_corrs = []
    for model_name in models:
        model_df = combined_df[combined_df['model'] == model_name]
        for clf_name in model_df['classifier'].unique():
            clf_data = model_df[model_df['classifier'] == clf_name]
            
            # Calculate Pearson correlation
            pearson_corr, p_value_pearson = pearsonr(
                clf_data['brier_score'].values, 
                clf_data['far'].values
            )
            
            # Calculate Spearman correlation
            spearman_corr, p_value_spearman = spearmanr(
                clf_data['brier_score'].values, 
                clf_data['far'].values
            )
            
            combined_corrs.append({
                'model': model_name,
                'classifier': clf_name,
                'pearson_correlation': pearson_corr,
                'pearson_p_value': p_value_pearson,
            })
    
    corr_df = pd.DataFrame(combined_corrs)
    corr_csv_path = 'brier_far_analysis/combined_brier_far_correlations.csv'
    corr_df.to_csv(corr_csv_path, index=False)
    print(f"Exported combined correlation results to {corr_csv_path}")
    
    # Create a summary plot for all models and classifiers
    plt.figure(figsize=(16, 12))
    
    # Colors for different models and markers for different classifiers
    model_colors = {'roberta': 'blue', 'modernbert': 'green', 'deberta': 'red', 'flair': 'purple'}
    clf_markers = {'svc': 'o', 'logistic_regression': 's', 'catboost': '^'}
    
    for model_name in models:
        model_df = combined_df[combined_df['model'] == model_name]
        for clf_name in model_df['classifier'].unique():
            clf_data = model_df[model_df['classifier'] == clf_name]
            
            plt.scatter(
                clf_data['brier_score'], 
                clf_data['far'],
                label=f"{model_name} - {clf_name}",
                color=model_colors.get(model_name, 'black'),
                marker=clf_markers.get(clf_name, 'x'),
                alpha=0.7,
                s=80
            )
    
    # Add labels and title
    plt.xlabel('Brier Score', fontsize=14)
    plt.ylabel('False Accept Rate (FAR)', fontsize=14)
    plt.title('Brier Score vs FAR - All Models and Classifiers', fontsize=16)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot
    plt_path = 'brier_far_analysis/all_models_classifiers_brier_vs_far.png'
    plt.savefig(plt_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved combined plot for all models to {plt_path}")
    
    print("\nAnalysis complete!")

if __name__ == "__main__":
    main()


============================== ANALYZING ROBERTA ==============================
Loaded embeddings from embeddings_output/roberta_PCA_150_components.csv

Evaluating svc classifier with roberta embeddings...
  Training scenario: baseline
    Testing against: baseline
    Testing against: gptv1
    Testing against: gptv2
    Testing against: gptv1v2
  Training scenario: gptv1_assisted
    Testing against: baseline
    Testing against: gptv1
    Testing against: gptv2
    Testing against: gptv1v2
  Training scenario: gptv2_assisted
    Testing against: baseline
    Testing against: gptv1
    Testing against: gptv2
    Testing against: gptv1v2
  Training scenario: gptv1v2_assisted
    Testing against: baseline
    Testing against: gptv1
    Testing against: gptv2
    Testing against: gptv1v2

Evaluating logistic_regression classifier with roberta embeddings...
  Training scenario: baseline
    Testing against: baseline
    Testing against: gptv1
    Testing against: gptv2
    Testing again